In [1]:
import os
os.getcwd()
os.chdir('D:\\Git projects\\college_works\\Thesis')

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from math import gamma
import time
import pandas as pd

In [2]:
def par():
    global beta, eta, varphi, theta, rho, i, r, gamma1, phi, kappa, alfa, psi, sig, pi, nu
    beta = 0.69
    eta = 0.25
    varphi = 0.25
    theta = 3.44
    rho = 0.19
    alfa = 0.6
    kappa = np.divide(1, (1- eta) )
    nu = 1 + np.multiply(alfa, np.multiply(varphi, kappa)) - np.divide(kappa, theta)
    pi = 1 - np.multiply(np.multiply( (1 - alfa), varphi), kappa )    
    sig = np.divide(np.multiply(eta, kappa), pi )
    psi = np.multiply(np.power(eta, eta), np.power( (1-eta), (1-eta) ) )
    i = 7
    r = 27
    gamma1 = gamma(   1 - np.multiply( np.divide(1, np.multiply(theta, (1-rho)) ), np.divide(1, (1 - eta) ) ) )   
    phi = np.array([0.138, 0.174, 0.136, 0.1, 0.051, 0.084, 0.168]).reshape(i, 1)

In [3]:

def taus2():
    par()
        
    tau_h = np.random.uniform(low=-0.99, high=40, size=(i,r))
    tau_h[0, :] = 0

    tau_w = np.random.uniform(low=-0.99, high=0.999, size=(i,r))
    tau_w[0, : ] = tau_w[0, 0]
    
    w =np.random.uniform(low=0.001, high=30, size=(i,r))
    w[0, r-1] = 1
    #w[1:6, :] = w[0, :] 
    
    x1 = np.array([tau_w, tau_h, w])
    
    return x1

x1 = taus2()

In [4]:
def sf( ):
    global s
    s = np.power( (1+ np.divide( (1-eta), ( np.multiply(beta, phi) ) ) ), -1 )
    s = s.reshape(i, 1)
    return s

In [5]:
def p_trf(x1):    
    s = sf( )    
    A = np.divide( (1 - x1[0]) , np.power( ( 1 + x1[1] ), eta) )     
    b = np.power(s, phi ) 
    B = np.multiply(b.reshape(i,1), x1[2] )     
    C = np.power( (1 - s), np.divide((1-eta), beta) )
    d = np.multiply(np.multiply(A, B), C )
    k = np.power(d, theta)
    p_tr = np.divide( k[i-1], np.sum(k, axis=0) )
    return p_tr
 

In [6]:
  
def H_trf(x1):    
    p_tr = p_trf(x1)  
    A = np.power(np.multiply( np.divide( (1 - x1[0]), (1 + x1[1]) ), x1[2] ), sig )    
    A = A[i-1]     
    P = np.multiply(np.power(p_tr, ( np.divide(nu, pi) )), np.power(eta, sig ) )
    g = np.divide(np.multiply(phi[i-1], kappa ), pi )
    C = np.multiply( np.power(s[i-1], g ), np.power(gamma1, np.divide(1, pi) ) )
    H_tr = np.multiply(np.multiply(P, A), C )       
    return H_tr

In [7]:
def w_tilf(x1):
    H_tr = H_trf(x1) 
    p_tr = p_trf(x1)                 
    C = np.power((1 - s), np.divide( (1-eta), beta )).reshape(7, 1)
    pp = np.power(s, phi)        
    A = np.multiply(psi, np.multiply( np.divide( (1 - x1[0]), np.power( (1 + x1[1]), eta ) ), x1[2] ) )    
    b = np.power(np.multiply(np.power(p_tr, alfa), np.power(H_tr, (1-alfa) ) ), varphi )    
    B = np.multiply(b, pp.reshape(7,1))
    w_til =  np.multiply(np.multiply(A, B), C )
    return w_til 

In [8]:

def p_irf(x1):    
    w_til = w_tilf(x1)
    w_til2 = np.power(w_til, theta) 
    w_r = w_til2.sum(axis = 0)    
    p_ir = np.divide(w_til2 , w_r ) 
    return np.array(p_ir), np.array(w_r)

In [9]:
def Wf(x1):
    p_ir, w_r = p_irf(x1)         
    z = np.multiply(np.multiply(gamma1, eta), np.power(w_r, np.divide(1, np.multiply(theta, (1 - eta)) ) ) )
    A = np.divide( np.power( (1 - s), (-1/beta) ), ( 1 - x1[0] )  )                
    W = np.multiply(A, z)
    return W, p_ir

In [10]:

def simul():
    global p_t, W_t
    p_t = pd.read_csv('pt.csv', sep=';')
    p_t = p_t.iloc[0:7]
    p_t.set_index('ocup', inplace=True)
    p_t = np.array(p_t)
    W_t = pd.read_csv('wt.csv', sep=';')
    W_t.set_index('ocup', inplace=True)
    W_t = np.array(W_t)
    
    return p_t, W_t

simul();

In [11]:


p_t2 = np.array(p_t[0:6])
    
def obj2(x1):       
    x1 = x1.reshape((3, i, r)) 
    x1[0, 0, : ] = x1[0, 0, 0]    
    x1[1, 0, :] = 0
    x1[2, :, 16] = 8
    x1[2, 0:7, :] = x1[2, 0, :]
    W, p_ir = Wf(x1)    
    p_ir = p_ir[0:6]
    D =   (np.power(np.divide( (W-W_t), W_t ), 2)).sum() + (np.power(np.divide( (p_ir-p_t2), p_t2 ), 2)).sum() 
    D = np.log(D)    
    return D

In [12]:
#%time obj2(np.vstack(taus2()).flatten())
%time obj2(taus2())

In [13]:
# optimization give me z1 array

z1 = pd.read_excel('MG_8.xlsx')
z1 = np.array(z1).reshape(3, i, r)


obj2(z1)

In [14]:
def h_tilf(x1):
    H_tr = H_trf(x1) 
    p_tr = p_trf(x1) 
    A = np.power(np.multiply(np.power(p_tr, alfa), np.power(H_tr, (1-alfa) ) ), varphi )    
    B = np.multiply(np.power(s, phi), np.power(eta, eta) )
    h_til = np.power(np.multiply(A, B), kappa )
    return h_til

In [15]:
def H_irf(x1):
    p_ir = p_irf(x1)[0]
    h_til = h_tilf(x1)
    A = np.multiply(h_til, np.power(p_ir, (1-kappa/theta) ) )
    B = np.power(np.multiply( np.divide( (1 - x1[0]), (1 + x1[1]) ), x1[2] ), np.multiply(eta, kappa ) )       
    H_ir = np.multiply(np.multiply(A, B), gamma1 )
    return H_ir
    
    

In [16]:


def Y_f(x1):
    H_ir = H_irf(x1) 
    Y = np.multiply(x1[2, 0, :], H_ir)
    return Y


    

In [17]:

# Model

W, p_ir = Wf(z1.reshape(3, i, r))
W = pd.DataFrame(W)
p_ir = pd.DataFrame(p_ir)

 
# PNAD data

p_t, W_t = simul()   
W_t =  pd.DataFrame(W_t)
p_t =  pd.DataFrame(p_t)


# get colnames

n = pd.read_csv('pt.csv', sep=';')
n = n.iloc[0:7]
n.set_index('ocup', inplace=True)
names = n.columns.str.strip("'")
names  = np.array(names).repeat(7) 



p_t = p_t.values.flatten() 
p_ir = p_ir.values.flatten()

W = W.values.flatten()
W_t = W_t.values.flatten()

In [18]:
# plots

### W and W_ir

W_t =  pd.DataFrame(W_t)
W = pd.DataFrame(W)


W = W.values.flatten()
W_t = W_t.values.flatten()


ocn1 = ['Managers','Sciences and arts','Middle-level technicians','Service-sector','Agriculture',
       'Industrial production and services','Teachers']
ocn1 = np.array(ocn1, dtype=object).repeat(27)

dd = pd.DataFrame(dict(x=W, y=W_t, label=ocn1))
groups = dd.groupby('label')


## p_ir and p_t

p_t =  pd.DataFrame(p_t)
p_ir = pd.DataFrame(p_ir)


p_t = p_t.values.flatten() 
p_ir = p_ir.values.flatten()

ocn = ['Managers','Sciences and arts','Middle-level technicians','Service-sector','Agriculture',
       'Industrial production and services','Teachers']


ocn = np.array(ocn, dtype=object).repeat(27)


ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
groups2 = ff.groupby('label')

In [19]:
z1

In [20]:
# tw, th, w, 

tau_w = z1[0]

In [21]:
# tw, th, w, 

tau_w = z1[0]

In [22]:
names

In [23]:
n

In [24]:
n = pd.read_csv('pt.csv', sep=';')
n = n.iloc[0:7]
n.set_index('ocup', inplace=True)
names2 = n.columns.str.strip("'")

In [25]:
# tw, th, w, 
n = pd.read_csv('pt.csv', sep=';')
n = n.iloc[0:7]
n.set_index('ocup', inplace=True)
names2 = n.columns.str.strip("'")


tau_w = pd.DataFrame(z1[0])
tau_w.columns = names2

In [26]:
# tw, th, w, 
n = pd.read_csv('pt.csv', sep=';')
n = n.iloc[0:7]
n.set_index('ocup', inplace=True)
names2 = n.columns.str.strip("'")


tau_w = pd.DataFrame(z1[0], index=ocn, columns= names2)

In [27]:
# tw, th, w, 
n = pd.read_csv('pt.csv', sep=';')
n = n.iloc[0:7]
n.set_index('ocup', inplace=True)
names2 = n.columns.str.strip("'")
ocup = ['Managers','Sciences and arts','Middle-level technicians','Service-sector','Agriculture',
       'Industrial production and services','Teachers']

tau_w = pd.DataFrame(z1[0], index=ocup, columns= names2)

In [28]:
z1[2]

In [29]:
z1[2][0]

In [30]:
z1[2,0]

In [31]:
# tw, th, w, 
n = pd.read_csv('pt.csv', sep=';')
n = n.iloc[0:7]
n.set_index('ocup', inplace=True)
names2 = n.columns.str.strip("'")
ocup = ['Managers','Sciences and arts','Middle-level technicians','Service-sector','Agriculture',
       'Industrial production and services','Teachers']

tau_w = pd.DataFrame(z1[0], index=ocup, columns= names2)
tau_h = pd.DataFrame(z1[1], index=ocup, columns= names2)
tfp = pd.DataFrame(z1[2,0], columns=names2)

In [32]:
# tw, th, w, 
n = pd.read_csv('pt.csv', sep=';')
n = n.iloc[0:7]
n.set_index('ocup', inplace=True)
names2 = n.columns.str.strip("'")
ocup = ['Managers','Sciences and arts','Middle-level technicians','Service-sector','Agriculture',
       'Industrial production and services','Teachers']

tau_w = pd.DataFrame(z1[0], index=ocup, columns= names2)
tau_h = pd.DataFrame(z1[1], index=ocup, columns= names2)
#tfp = pd.DataFrame(z1[2,0], columns=names2)

In [33]:
tfp = pd.DataFrame(z1[2,0], columns=names2)

In [34]:
#tfp = pd.DataFrame(z1[2,0], columns=names2)
names2

In [35]:
tfp = pd.DataFrame(z1[2,0], columns=names2)
#names2

In [36]:
#tfp = pd.DataFrame(z1[2,0], columns=names2)
#names2

z1[2,0].shape()

In [37]:
#tfp = pd.DataFrame(z1[2,0], columns=names2)
#names2

z1[2,0].shape

In [38]:
#tfp = pd.DataFrame(z1[2,0], columns=names2)
#names2

z1[2][0].shape

In [39]:
#tfp = pd.DataFrame(z1[2,0], columns=names2)
#names2

z1[2][0].reshape(27)

In [40]:
#tfp = pd.DataFrame(z1[2,0], columns=names2)
#names2

z1[2][0].reshape(27).shape()

In [41]:
#tfp = pd.DataFrame(z1[2,0], columns=names2)
#names2

z1[2][0].reshape(27).shape

In [42]:
#tfp = pd.DataFrame(z1[2,0], columns=names2)
#names2

z1[2][0].reshape(1, 27).shape

In [43]:
tfp = pd.DataFrame(z1[2][0].reshape(1, 27), columns=names2)
#names2

In [44]:
tau_w

In [45]:
print(tau_w.to_latex(index=True))

In [46]:
print(tau_w.round(4).to_latex(index=True))

In [47]:
print(tau_w.round(4).transpose().to_latex(index=True))

In [48]:
print(tau_h.round(4).transpose().to_latex(index=True))

In [49]:
print(tfp.round(4).transpose().to_latex(index=True))

In [50]:
#print(tfp.round(4).transpose().to_latex(index=True))

In [51]:
    
fig, ax = plt.subplots(1, 2, figsize=(14,8))  

ax[0].plot(bb, pct(Htr_th), label=r'$H_{tr}$', color='blue', marker='o')
ax[0].tick_params(axis='both', which='major', labelsize=20)
ax[0].grid(True)
ax[0].set_xlabel(r"Increases in $\tau^h_{tr}$ ", fontsize=20)
ax[0].set_ylabel('Percentage change in $H_{tr}$', fontsize=20)
ax[0].legend(loc="lower left", prop={'size': 20})
ax[0].set_title('(a)', fontsize=20)
ax[0].set_xlim([0, 0.6])
ax2 =ax[0].twinx()
ax2.plot(bb, pct(ptr_th),  label=r'$p_{tr}$', color='red', marker='x')
ax2.set_ylabel(r"Percentage change in $p_{tr}$", fontsize=18)
plt.legend(loc="upper right", prop={'size': 16})
plt.yticks(fontsize=20)





ax[1].plot(bb, pct(Htr_tw), label=r'$H_{tr}$', color='blue', marker='o')
ax[1].tick_params(axis='both', which='major', labelsize=20)
ax[1].set_xlabel(r"Increases in $\tau^w_{tr}$ ", fontsize=20)
ax[1].set_ylabel('Percentage change in $H_{tr}$', fontsize=20)
ax[1].grid(True)
ax[1].legend(loc="lower left", prop={'size': 20})
ax[1].set_title('(b)', fontsize=20)
ax[1].set_xlim([0, 0.6])
ax3 =ax[1].twinx()
ax3.plot(bb, pct(ptr_tw),  label=r'$p_{tr}$', color='red', marker='x')
ax3.set_ylabel(r"Percentage change in $p_{tr}$", fontsize=18)
plt.legend(loc="upper right", prop={'size': 16})
plt.yticks(fontsize=20)

plt.tight_layout()    
#plt.savefig("D:/DOUTORADO/Topics in macro/paper_tex/F2.eps", format='eps', dpi=1200)

In [52]:
# increases in t^w

par()

bb = np.arange(0, 0.61, 0.05)
n = len(bb) 
Htr_tw = []

ptr_tw = []
    
for nn in bb: 
    mm = np.array(z1, copy=True)
    mm[0, i-1, :] = mm[0, i-1, :] + nn
    c1 = mm[0]
    c2 = mm[1] 
    c3 = mm[2]
    C = np.array([c1, c2, c3])
    Htr_tw.append(H_trf(mm).sum()) 
    ptr_tw.append(p_irf(mm)[0].sum(axis=1)[-1]/p_irf(mm)[0].sum(axis=1).sum()) 
    

    
# increases in t^w

bb = np.arange(0, 0.61, 0.05)
n = len(bb) 
Htr_th = []

ptr_th = []
    
for nn in bb: 
    mm = np.array(z1, copy=True)
    mm[1, i-1, :] = mm[1, i-1, :] + nn
    c1 = mm[0]
    c2 = mm[1] 
    c3 = mm[2]
    C = np.array([c1, c2, c3])
    Htr_th.append(H_trf(mm).sum()) 
    ptr_th.append( p_irf(mm)[0].sum(axis=1)[-1]/p_irf(mm)[0].sum(axis=1).sum()) 
    

In [53]:
    
fig, ax = plt.subplots(1, 2, figsize=(14,8))  

ax[0].plot(bb, pct(Htr_th), label=r'$H_{tr}$', color='blue', marker='o')
ax[0].tick_params(axis='both', which='major', labelsize=20)
ax[0].grid(True)
ax[0].set_xlabel(r"Increases in $\tau^h_{tr}$ ", fontsize=20)
ax[0].set_ylabel('Percentage change in $H_{tr}$', fontsize=20)
ax[0].legend(loc="lower left", prop={'size': 20})
ax[0].set_title('(a)', fontsize=20)
ax[0].set_xlim([0, 0.6])
ax2 =ax[0].twinx()
ax2.plot(bb, pct(ptr_th),  label=r'$p_{tr}$', color='red', marker='x')
ax2.set_ylabel(r"Percentage change in $p_{tr}$", fontsize=18)
plt.legend(loc="upper right", prop={'size': 16})
plt.yticks(fontsize=20)





ax[1].plot(bb, pct(Htr_tw), label=r'$H_{tr}$', color='blue', marker='o')
ax[1].tick_params(axis='both', which='major', labelsize=20)
ax[1].set_xlabel(r"Increases in $\tau^w_{tr}$ ", fontsize=20)
ax[1].set_ylabel('Percentage change in $H_{tr}$', fontsize=20)
ax[1].grid(True)
ax[1].legend(loc="lower left", prop={'size': 20})
ax[1].set_title('(b)', fontsize=20)
ax[1].set_xlim([0, 0.6])
ax3 =ax[1].twinx()
ax3.plot(bb, pct(ptr_tw),  label=r'$p_{tr}$', color='red', marker='x')
ax3.set_ylabel(r"Percentage change in $p_{tr}$", fontsize=18)
plt.legend(loc="upper right", prop={'size': 16})
plt.yticks(fontsize=20)

plt.tight_layout()    
#plt.savefig("D:/DOUTORADO/Topics in macro/paper_tex/F2.eps", format='eps', dpi=1200)

In [54]:
def pct(a):
   return ((a - a[0])/a[0] )*100

In [55]:
    
fig, ax = plt.subplots(1, 2, figsize=(14,8))  

ax[0].plot(bb, pct(Htr_th), label=r'$H_{tr}$', color='blue', marker='o')
ax[0].tick_params(axis='both', which='major', labelsize=20)
ax[0].grid(True)
ax[0].set_xlabel(r"Increases in $\tau^h_{tr}$ ", fontsize=20)
ax[0].set_ylabel('Percentage change in $H_{tr}$', fontsize=20)
ax[0].legend(loc="lower left", prop={'size': 20})
ax[0].set_title('(a)', fontsize=20)
ax[0].set_xlim([0, 0.6])
ax2 =ax[0].twinx()
ax2.plot(bb, pct(ptr_th),  label=r'$p_{tr}$', color='red', marker='x')
ax2.set_ylabel(r"Percentage change in $p_{tr}$", fontsize=18)
plt.legend(loc="upper right", prop={'size': 16})
plt.yticks(fontsize=20)





ax[1].plot(bb, pct(Htr_tw), label=r'$H_{tr}$', color='blue', marker='o')
ax[1].tick_params(axis='both', which='major', labelsize=20)
ax[1].set_xlabel(r"Increases in $\tau^w_{tr}$ ", fontsize=20)
ax[1].set_ylabel('Percentage change in $H_{tr}$', fontsize=20)
ax[1].grid(True)
ax[1].legend(loc="lower left", prop={'size': 20})
ax[1].set_title('(b)', fontsize=20)
ax[1].set_xlim([0, 0.6])
ax3 =ax[1].twinx()
ax3.plot(bb, pct(ptr_tw),  label=r'$p_{tr}$', color='red', marker='x')
ax3.set_ylabel(r"Percentage change in $p_{tr}$", fontsize=18)
plt.legend(loc="upper right", prop={'size': 16})
plt.yticks(fontsize=20)

plt.tight_layout()    
#plt.savefig("D:/DOUTORADO/Topics in macro/paper_tex/F2.eps", format='eps', dpi=1200)

In [56]:
    
fig, ax = plt.subplots(1, 2, figsize=(14,8))  

ax[0].plot(bb, pct(Htr_th), label=r'$H_{tr}$', color='blue', marker='o')
ax[0].tick_params(axis='both', which='major', labelsize=20)
ax[0].grid(True)
ax[0].set_xlabel(r"Increases in $\tau^h_{tr}$ ", fontsize=20)
ax[0].set_ylabel('Percentage change$', fontsize=20)
ax[0].legend(loc="lower left", prop={'size': 20})
ax[0].set_title('(a)', fontsize=20)
ax[0].set_xlim([0, 0.6])
ax[0].plot(bb, pct(ptr_th),  label=r'$p_{tr}$', color='red', marker='x')
plt.legend(loc="upper right", prop={'size': 16})
#plt.yticks(fontsize=20)





ax[1].plot(bb, pct(Htr_tw), label=r'$H_{tr}$', color='blue', marker='o')
ax[1].tick_params(axis='both', which='major', labelsize=20)
ax[1].set_xlabel(r"Increases in $\tau^w_{tr}$ ", fontsize=20)
ax[1].set_ylabel('Percentage change in $H_{tr}$', fontsize=20)
ax[1].grid(True)
ax[1].legend(loc="lower left", prop={'size': 20})
ax[1].set_title('(b)', fontsize=20)
ax[1].set_xlim([0, 0.6])
ax3 =ax[1].twinx()
ax3.plot(bb, pct(ptr_tw),  label=r'$p_{tr}$', color='red', marker='x')
ax3.set_ylabel(r"Percentage change in $p_{tr}$", fontsize=18)
plt.legend(loc="upper right", prop={'size': 16})
plt.yticks(fontsize=20)

plt.tight_layout()    
#plt.savefig("D:/DOUTORADO/Topics in macro/paper_tex/F2.eps", format='eps', dpi=1200)

In [57]:
    
fig, ax = plt.subplots(1, 2, figsize=(14,8))  

ax[0].plot(bb, pct(Htr_th), label=r'$H_{tr}$', color='blue', marker='o')
ax[0].plot(bb, pct(ptr_th),  label=r'$p_{tr}$', color='red', marker='X')
ax[0].tick_params(axis='both', which='major', labelsize=20)
ax[0].grid(True)
ax[0].set_xlabel(r"Increases in $\tau^h_{tr}$ ", fontsize=20)
ax[0].set_ylabel('Percentage change', fontsize=20)
ax[0].legend(loc="lower left", prop={'size': 20})
ax[0].set_title('(a)', fontsize=20)
ax[0].set_xlim([0, 0.6])
plt.legend(loc="upper right", prop={'size': 16})
#plt.yticks(fontsize=20)





ax[1].plot(bb, pct(Htr_tw), label=r'$H_{tr}$', color='blue', marker='o')
ax[1].tick_params(axis='both', which='major', labelsize=20)
ax[1].set_xlabel(r"Increases in $\tau^w_{tr}$ ", fontsize=20)
ax[1].set_ylabel('Percentage change in $H_{tr}$', fontsize=20)
ax[1].grid(True)
ax[1].legend(loc="lower left", prop={'size': 20})
ax[1].set_title('(b)', fontsize=20)
ax[1].set_xlim([0, 0.6])
ax3 =ax[1].twinx()
ax3.plot(bb, pct(ptr_tw),  label=r'$p_{tr}$', color='red', marker='x')
ax3.set_ylabel(r"Percentage change in $p_{tr}$", fontsize=18)
plt.legend(loc="upper right", prop={'size': 16})
plt.yticks(fontsize=20)

plt.tight_layout()    
#plt.savefig("D:/DOUTORADO/Topics in macro/paper_tex/F2.eps", format='eps', dpi=1200)

In [58]:
    
fig, ax = plt.subplots(1, 2, figsize=(14,8))  

ax[0].plot(bb, pct(Htr_th), label=r'$H_{tr}$', color='blue', marker='o')
ax[0].plot(bb, pct(ptr_th),  label=r'$p_{tr}$', color='red', marker='s')
ax[0].tick_params(axis='both', which='major', labelsize=20)
ax[0].grid(True)
ax[0].set_xlabel(r"Increases in $\tau^h_{tr}$ ", fontsize=20)
ax[0].set_ylabel('Percentage change', fontsize=20)
ax[0].legend(loc="lower left", prop={'size': 20})
ax[0].set_title('(a)', fontsize=20)
ax[0].set_xlim([0, 0.6])
plt.legend(loc="upper right", prop={'size': 16})
#plt.yticks(fontsize=20)





ax[1].plot(bb, pct(Htr_tw), label=r'$H_{tr}$', color='blue', marker='o')
ax[1].plot(bb, pct(ptr_tw),  label=r'$p_{tr}$', color='red', marker='s')
ax[1].tick_params(axis='both', which='major', labelsize=20)
ax[1].set_xlabel(r"Increases in $\tau^w_{tr}$ ", fontsize=20)
ax[1].set_ylabel('Percentage change', fontsize=20)
ax[1].grid(True)
ax[1].legend(loc="lower left", prop={'size': 20})
ax[1].set_title('(b)', fontsize=20)
ax[1].set_xlim([0, 0.6])
plt.legend(loc="upper right", prop={'size': 16})

plt.tight_layout()    
#plt.savefig("D:/DOUTORADO/Topics in macro/paper_tex/F2.eps", format='eps', dpi=1200)

In [59]:
    
fig, ax = plt.subplots(1, 2, figsize=(14,8))  

ax[0].plot(bb, pct(Htr_th), label=r'$H_{tr}$', color='blue', marker='o')
ax[0].plot(bb, pct(ptr_th),  label=r'$p_{tr}$', color='red', marker='s')
ax[0].tick_params(axis='both', which='major', labelsize=20)
ax[0].grid(True)
ax[0].set_xlabel(r"Increases in $\tau^h_{tr}$ ", fontsize=20)
ax[0].set_ylabel('Percentage change', fontsize=20)
ax[0].legend(loc="lower left", prop={'size': 20})
ax[0].set_title('(a)', fontsize=20)
ax[0].set_xlim([0, 0.6])
#plt.yticks(fontsize=20)





ax[1].plot(bb, pct(Htr_tw), label=r'$H_{tr}$', color='blue', marker='o')
ax[1].plot(bb, pct(ptr_tw),  label=r'$p_{tr}$', color='red', marker='s')
ax[1].tick_params(axis='both', which='major', labelsize=20)
ax[1].set_xlabel(r"Increases in $\tau^w_{tr}$ ", fontsize=20)
ax[1].set_ylabel('Percentage change', fontsize=20)
ax[1].grid(True)
ax[1].legend(loc="lower left", prop={'size': 20})
ax[1].set_title('(b)', fontsize=20)
ax[1].set_xlim([0, 0.6])

plt.tight_layout()    
#plt.savefig("D:/DOUTORADO/Topics in macro/paper_tex/F2.eps", format='eps', dpi=1200)

In [60]:
    
fig, ax = plt.subplots(1, 2, figsize=(14,8))  

ax[0].plot(bb, pct(Htr_th), label=r'$H_{tr}$', color='blue', marker='o')
ax[0].plot(bb, pct(ptr_th),  label=r'$p_{tr}$', color='red', marker='s')
ax[0].tick_params(axis='both', which='major', labelsize=20)
ax[0].grid(True)
ax[0].set_xlabel(r"Increases in $\tau^h_{tr}$ ", fontsize=20)
ax[0].set_ylabel('Percentage change', fontsize=20)
ax[0].legend(loc="upper right", prop={'size': 20})
ax[0].set_title('(a)', fontsize=20)
ax[0].set_xlim([0, 0.6])
#plt.yticks(fontsize=20)





ax[1].plot(bb, pct(Htr_tw), label=r'$H_{tr}$', color='blue', marker='o')
ax[1].plot(bb, pct(ptr_tw),  label=r'$p_{tr}$', color='red', marker='s')
ax[1].tick_params(axis='both', which='major', labelsize=20)
ax[1].set_xlabel(r"Increases in $\tau^w_{tr}$ ", fontsize=20)
ax[1].set_ylabel('Percentage change', fontsize=20)
ax[1].grid(True)
ax[1].legend(loc="upper right", prop={'size': 20})
ax[1].set_title('(b)', fontsize=20)
ax[1].set_xlim([0, 0.6])

plt.tight_layout()    
#plt.savefig("D:/DOUTORADO/Topics in macro/paper_tex/F2.eps", format='eps', dpi=1200)

In [61]:
    
fig, ax = plt.subplots(1, 2, figsize=(14,8))  

ax[0].plot(bb, pct(Htr_th), label=r'$H_{tr}$', color='blue', marker='o')
ax[0].plot(bb, pct(ptr_th),  label=r'$p_{tr}$', color='red', marker='s')
ax[0].tick_params(axis='both', which='major', labelsize=20)
ax[0].grid(True)
ax[0].set_xlabel(r"Increases in $\tau^h_{tr}$ ", fontsize=20)
ax[0].set_ylabel('Percentage change', fontsize=20)
ax[0].legend(loc="upper right", prop={'size': 20})
ax[0].set_title('(a)', fontsize=20)
ax[0].set_xlim([0, 0.6])
#plt.yticks(fontsize=20)





ax[1].plot(bb, pct(Htr_tw), label=r'$H_{tr}$', color='blue', marker='o')
ax[1].plot(bb, pct(ptr_tw),  label=r'$p_{tr}$', color='red', marker='s')
ax[1].tick_params(axis='both', which='major', labelsize=20)
ax[1].set_xlabel(r"Increases in $\tau^w_{tr}$ ", fontsize=20)
ax[1].set_ylabel('Percentage change', fontsize=20)
ax[1].grid(True)
ax[1].legend(loc="upper right", prop={'size': 20})
ax[1].set_title('(b)', fontsize=20)
ax[1].set_xlim([0, 0.6])

plt.tight_layout()    
plt.savefig("D:/DOUTORADO/Topics in macro/paper_tex/F2.eps", format='eps', dpi=1200)